In [8]:
from nltk import PCFG
from nltk.parse.generate import generate
import random

import numpy
class Stack:
    def __init__(self):
        self.__storage = []

    def isEmpty(self):
        return len(self.__storage) == 0

    def push(self,p):
        self.__storage.append(p)

    def pop(self):
        return self.__storage.pop()

In [9]:
branch_at_height = [3, 3, 3, 3]
# len = len (branch_at_height) + 1
probs_tree_mut = [0.8, 0.8, 0.8, 0.8, 0.8]

# the marker insertion, deletion probs, n_type of marker
marker_prob = {'mrkr1': (0.05, 0.005, 3), 'mrkr2': (0.05, 0.005, 5), 'mrkr3': (0.05, 0.005, 2)}

# how phenotypes are related to the important mutations
phenotype_marker = {'drug1': ['mrkr1'],'drug2': ['mrkr2'],'drug3': ['mrkr3']}

grammar_rules=[]

grammar_dictionary=dict()

non_terminals_at_previous_branch = ['S']

for idx, branch_width in enumerate(branch_at_height):
    prob_tree_mut = probs_tree_mut[idx]
    temp_current_branch = []
    for non_terminal in non_terminals_at_previous_branch:
        # generate all non_terminals of this branch
        non_terminals_at_current_branch = [non_terminal+str(i+1) for i in range(branch_width)]
        # generate terminal rule
        terminal_ruls_at_current_branch = [([F"'phlgntp{non_terminal[1::]}'", non_terminal], prob_tree_mut)]
        # generate marker related rule
        p_markers = 0
        insert_marker_rules = []
        delete_marker_rules = []
        for mutation, (p_ins, p_del,_) in marker_prob.items():
            insert_marker_rules.append(([F"'{mutation}'", non_terminal], p_ins))
            delete_marker_rules.append(([F"'del_{mutation}'", non_terminal], p_del))
            p_markers = p_markers + p_ins + p_del
        p = 1 - prob_tree_mut - p_markers
        # generate the rule
        trans = [([S],p/branch_width) for S in non_terminals_at_current_branch]
        list_of_rhs=[' '.join(lrhs+[F"[{p}]"]) for lrhs,p  in terminal_ruls_at_current_branch + insert_marker_rules+delete_marker_rules+trans]
        grammar_rules.append(F"{non_terminal}  -> {' | '.join(list_of_rhs)}")

        list_of_p=[p for lrhs,p  in terminal_ruls_at_current_branch + insert_marker_rules+delete_marker_rules+trans]
        list_of_lrhs=[lrhs for lrhs,p  in terminal_ruls_at_current_branch + insert_marker_rules+delete_marker_rules+trans]                     
        grammar_dictionary [non_terminal] = (list_of_p, list_of_lrhs)                    
        
        temp_current_branch = temp_current_branch + non_terminals_at_current_branch
    non_terminals_at_previous_branch = temp_current_branch

for non_terminal in non_terminals_at_previous_branch:
    prob_tree_mut = probs_tree_mut[-1]
    # generate marker related rule
    p_markers = 0
    insert_marker_rules = []
    delete_marker_rules = []
    for mutation, (p_ins, p_del,_) in marker_prob.items():
        insert_marker_rules.append(([F"'{mutation}'", non_terminal], p_ins))
        delete_marker_rules.append(([F"'del_{mutation}'", non_terminal], p_del))
        p_markers = p_markers + p_ins + p_del
    prob_end = 1 - prob_tree_mut - p_markers                     
    terminal_ruls_at_current_branch = ([F"'phlgntp{non_terminal[1::]}'", non_terminal], prob_tree_mut)                             
    terminate_rule=([F"'phlgntp{non_terminal[1::]}"],prob_end)                      
    
    list_of_rhs=[' '.join(lrhs+[F"[{p}]"]) for lrhs,p  in [terminal_ruls_at_current_branch,terminate_rule] + insert_marker_rules+delete_marker_rules]
    grammar_rules.append(F"{non_terminal}  -> {' | '.join(list_of_rhs)}")

    list_of_lrhs=[lrhs for lrhs,p  in [terminal_ruls_at_current_branch,terminate_rule] + insert_marker_rules+delete_marker_rules]
    list_of_p=[p for lrhs,p  in [terminal_ruls_at_current_branch,terminate_rule] + insert_marker_rules+delete_marker_rules]
                         
    grammar_dictionary [non_terminal] = (list_of_p, list_of_lrhs)                    

#pcfg = PCFG.fromstring('\n'.join(grammar_rules))
#pcfg_prods = pcfg.productions()
                       

In [10]:
grammar_dictionary

{'S': ([0.8,
   0.05,
   0.05,
   0.05,
   0.005,
   0.005,
   0.005,
   0.01166666666666664,
   0.01166666666666664,
   0.01166666666666664],
  [["'phlgntp'", 'S'],
   ["'mrkr1'", 'S'],
   ["'mrkr2'", 'S'],
   ["'mrkr3'", 'S'],
   ["'del_mrkr1'", 'S'],
   ["'del_mrkr2'", 'S'],
   ["'del_mrkr3'", 'S'],
   ['S1'],
   ['S2'],
   ['S3']]),
 'S1': ([0.8,
   0.05,
   0.05,
   0.05,
   0.005,
   0.005,
   0.005,
   0.01166666666666664,
   0.01166666666666664,
   0.01166666666666664],
  [["'phlgntp1'", 'S1'],
   ["'mrkr1'", 'S1'],
   ["'mrkr2'", 'S1'],
   ["'mrkr3'", 'S1'],
   ["'del_mrkr1'", 'S1'],
   ["'del_mrkr2'", 'S1'],
   ["'del_mrkr3'", 'S1'],
   ['S11'],
   ['S12'],
   ['S13']]),
 'S2': ([0.8,
   0.05,
   0.05,
   0.05,
   0.005,
   0.005,
   0.005,
   0.01166666666666664,
   0.01166666666666664,
   0.01166666666666664],
  [["'phlgntp2'", 'S2'],
   ["'mrkr1'", 'S2'],
   ["'mrkr2'", 'S2'],
   ["'mrkr3'", 'S2'],
   ["'del_mrkr1'", 'S2'],
   ["'del_mrkr2'", 'S2'],
   ["'del_mrkr3'", 'S2'

In [11]:
from nltk import FreqDist

def remove_values_from_list(the_list, val):
    return [value for value in the_list if value != val]

def polish_the_sample(sample, max_genotype_per_node, marker_prob, phenotype_marker):
    item_freq = FreqDist(sample).most_common()
    contained_markers = []
    for item, freq in item_freq:
        if item[1:8]=='phlgntp':
            new_items = list(set(['_'.join([item.replace("'",''),'_',str(idx+1)]) for idx in random.sample(list(range(max_genotype_per_node)),min(freq,max_genotype_per_node))]))
            sample = remove_values_from_list(sample, item)
            sample = sample + new_items

        if item[1:4]=='del':
            marker_to_delete = item.replace("'",'').split('_')[1]
            sample = remove_values_from_list(sample, F"'{marker_to_delete}'")
            sample = remove_values_from_list(sample, F"'del_{marker_to_delete}'")

    item_freq = FreqDist(sample).most_common()
    for item, freq in item_freq:
        if item[1:5]=='mrkr':
            marker=item.replace("'",'')
            contained_markers.append(marker)
            new_items = list(set(['_'.join([marker,'_',str(idx+1)]) for idx in random.sample(list(range(marker_prob[marker][2])),min(freq,marker_prob[marker][2]))]))
            sample = remove_values_from_list(sample, item)
            sample = sample + new_items
    
    phenotypes = []
    for phenotype, markers in phenotype_marker.items():
        if len(set(markers).intersection(contained_markers)):
            phenotypes.append(phenotype)
        
    return [sample, phenotypes]

In [12]:

def get_sample(grammar_dictionary,marker_prob,phenotype_marker, max_genotype_per_node = 20):

    sample_generate_stack = Stack()

    sample_generate_stack.push('S')

    sample = []

    while not sample_generate_stack.isEmpty():
        current_symbol = sample_generate_stack.pop()
        prob, symb_list = grammar_dictionary[current_symbol]
        try:
            symbols_idx = numpy.random.choice(range(len(symb_list)), p=prob)
        except:
            print(prob, symb_list)
        symbols = symb_list[symbols_idx]
        for x in symbols:
            if x in grammar_dictionary:
                sample_generate_stack.push(x)
            else:
                sample.append(x)
    return polish_the_sample(sample, 20, marker_prob, phenotype_marker)


In [67]:
samples=[]
labels = []
for x in range(100):
    a,b=get_sample(grammar_dictionary,marker_prob,phenotype_marker, max_genotype_per_node = 20)
    samples.append(a)
    labels.append(b)

In [68]:
f = open('genotype_table.txt','w')
for idx, sample in enumerate(samples):
    f.write(' '.join([F"ISO{str(idx+1)}"]+sample)+'\n')
f.close()

In [69]:
df = {'iso':[]}
df.update({x:[] for x in list(phenotype_marker.keys())})

for idx, label in enumerate(labels):
    df['iso'].append(F"ISO{str(idx+1)}")
    for l in list(phenotype_marker.keys()):
        if l in label:
            df[l].append(1)
        else:
            df[l].append(0)
import pandas as pd

pd.DataFrame(df).to_csv('phenotypes.txt',sep='\t',index=False)

In [70]:
phylogene_dict = dict()
phylogene_rev_dict = dict()

for idx, sample in enumerate(samples):
    phlyo = str(max([int(x.split('_')[0].replace('phlgntp','')) if x.split('_')[0].replace('phlgntp','').isdigit() else -1 for x in sample if 'phlgntp' in x]))
    phylogene_dict[F"ISO{str(idx+1)}"]=phlyo
    if phlyo not in phylogene_rev_dict:
        phylogene_rev_dict[phlyo]=[F"ISO{str(idx+1)}"]
    else:
        phylogene_rev_dict[phlyo].append(F"ISO{str(idx+1)}")
        

In [72]:
def nth_repl(s, sub, repl, nth):
    find = s.find(sub)
    # if find is not p1 we have found at least one match for the substring
    i = find != -1
    # loop util we find the nth or we find no match
    while find != -1 and i != nth:
        # find + 1 means we start at the last match start index + 1
        find = s.find(sub, find + 1)
        i += 1
    # if i  is equal to nth we found nth matches so replace
    if i == nth:
        return s[:find]+repl+s[find + len(sub):]
    return s

In [74]:
sizes = branch_at_height
tree = '(x)'
possiblities =[]
for x in sizes[::-1]:
    if possiblities ==[]:
        for i in range(1,x+1):
            possiblities.append(str(i))
    else:
        temp = possiblities
        new =  []
        for i in range(1,x+1):
            new=new+[str(i)+y for y in temp]
        possiblities = new
    
    tree = '('+','.join([tree]*x)+')'

for idx, pos in enumerate(possiblities[::-1]):
    tree = nth_repl(tree,'x',pos,len(possiblities)-idx)
    
for k,v in phylogene_rev_dict.items():
    tree=tree.replace(k,','.join(v))

In [82]:
f = open('phylo_tree.txt','w')

f.write(tree)
f.close()
